# Brain Tumor Detection
## Resnet101 - Classifier + Regressor
Description
This dataset was originally created by Yousef Ghanem. To see the current project, which may have been updated since this version, please go here: https://universe.roboflow.com/yousef-ghanem-jzj4y/brain-tumor-detection-fpf1f.

This dataset is part of RF100, an Intel-sponsored initiative to create a new object detection benchmark for model generalizability.

Access the RF100 Github repo: https://github.com/roboflow-ai/roboflow-100-benchmark

## Imports

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/brain-tumor-detection'

In [2]:
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

import tensorflow as tf
tf.random.set_seed(42)

import matplotlib.pyplot as plt

2025-03-24 07:56:03.666137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742802963.674251   48909 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742802963.676712   48909 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742802963.686598   48909 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742802963.686613   48909 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742802963.686615   48909 computation_placer.cc:177] computation placer alr

In [3]:
# Mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [4]:
found_gpu = tf.config.list_physical_devices('GPU')
if not found_gpu:
    raise Exception("No GPU found")
found_gpu, tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.19.0')

In [5]:
from src.data_handler.data_loader import DataLoader
from src.data_handler.annotation_processor import AnnotationProcessor
from src.data_handler.preprocessor import Preprocessor

In [6]:
# auto reload dotenv 
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

## Paths Setup

In [7]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/-Brain-Tumor-Detection-2/train/


In [8]:
cfg.DATASET_DIRS

{'TRAIN_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/train/', 'VALIDATION_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/valid', 'TEST_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/test'}

In [9]:
DATASET_DIRS = Path(cfg.DATASET.DATASET_DIR)
TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
VALIDATION_DIR = Path(cfg.DATASET_DIRS.VALIDATION_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)
OUTPUT_DIR = Path(cfg.OUTPUTS.OUTPUT_DIR)

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE
LOG_DIR = cfg.OUTPUTS.LOG_DIR
CHECK_POINT_DIR = Path(cfg.OUTPUTS.CHECKPOINT_PATH)
CLASS_NAME = [
    'label0',
    'label1',
    'label2'
]
class_map = {k: v for k, v in enumerate(CLASS_NAME)}

NUM_EPOCHS = cfg.TRAIN.NUM_EPOCHS
LEARNING_RATE = cfg.TRAIN.LEARNING_RATE

NUM_CLASSES = len(CLASS_NAME)

## Dataset Download from Roboflow

In [10]:
if not TRAIN_DIR.exists():
    from roboflow import Roboflow
    rf = Roboflow()
    project = rf.workspace("yousef-ghanem-jzj4y").project("brain-tumor-detection-fpf1f")
    version = project.version(2)
    dataset = version.download("tensorflow") 

## Load images from directory

In [11]:
prepare_train_dataset = AnnotationProcessor(annotation_file= str(TRAIN_DIR/'_annotations.csv'))
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
train_images, train_class_ids, train_bboxes  = prepare_train_dataset.process_annotations(image_dir=TRAIN_DIR, class_id_map=_class_map)

len(train_images), len(train_class_ids), len(train_bboxes)

(6851, 6851, 6851)

In [12]:
train_bboxes[0]

array([[0.68345324, 0.54545455, 0.95683453, 0.76515152],
       [0.42446043, 0.48484848, 0.99280576, 0.96969697],
       [0.46043165, 0.53030303, 0.99280576, 0.78030303]])

In [13]:
train_class_ids

[[0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1],
 [0, 1, 2],
 [0, 1, 2],
 [1, 2],
 [0, 1],
 [1],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [1, 2],
 [0, 1],
 [2],
 [1],
 [0, 1, 2],
 [1],
 [1],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1],
 [1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1],
 [0, 1, 2],
 [1],
 [0, 1],
 [0, 1, 2],
 [1],
 [1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [1],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2]

In [14]:
train_dl = DataLoader(train_images, train_class_ids, train_bboxes)
train_ds = train_dl.load_train_dataset()
train_ds = Preprocessor(train_ds).preprocess()
train_ds = train_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

I0000 00:00:1742802977.231147   48909 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7397 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6


In [15]:
for batch in train_ds.take(1):
    image, (cls, bbx) = batch
    print(image.shape, cls.shape, bbx.shape)
    print(cls[5])
    print(image[1].numpy().min(), image[1].numpy().max())
    for c in cls:
        print(c.numpy())

(32, 240, 240, 3) (32, 3) (32, 3, 4)
tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)
-123.7 145.0
[0. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[0. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]


2025-03-24 07:56:25.306497: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Validation datasets setup

In [16]:
prepare_valid_dataset = AnnotationProcessor(annotation_file= str(VALIDATION_DIR/'_annotations.csv'))

valid_image_paths, valid_class_ids, valid_bboxes  = prepare_valid_dataset.process_annotations(image_dir=VALIDATION_DIR, class_id_map=_class_map)
len(valid_image_paths), len(valid_class_ids), len(valid_bboxes)

(1963, 1963, 1963)

In [17]:
valid_dl = DataLoader(valid_image_paths, valid_class_ids, valid_bboxes).load_val_dataset()
valid_ds = Preprocessor(valid_dl).preprocess()
valid_ds = valid_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

In [18]:
for batch in valid_ds.take(1):
    image, (cls, bbx) = batch
    print(image.shape, cls.shape, bbx.shape)
    print(image[1].numpy().min(), image[1].numpy().max())

(32, 240, 240, 3) (32, 3) (32, 3, 4)
-123.68 138.49847


2025-03-24 07:56:26.487140: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Training Setup

## Define loss

In [19]:
padded_class_ids = train_dl.multi_hot_class_ids
padded_class_ids[:10]

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 0.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [20]:
from src.losses import binary_weighted_loss as _loss

positive_weights, negative_weights = _loss.compute_class_weights(padded_class_ids)
positive_weights, negative_weights 

(array([0.38140417, 0.01547219, 0.40767771]),
 array([0.61859584, 0.9845278 , 0.5923223 ], dtype=float32))

## Define ResNet50 Model

### Final Model

In [21]:
from src.losses import iou_loss
CLS_METRICS = [
    tf.keras.metrics.AUC(name='AUC', multi_label=True), 
    tf.keras.metrics.F1Score(name='f1_score',average='weighted')]


REG_METRICS = [
    iou_loss.iou_metric,
    tf.keras.metrics.MeanSquaredError(name='mse'),
    tf.keras.metrics.MeanAbsoluteError(name='mae'),]

### Define  Callbacks

In [22]:
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, TensorSpec
# 1. Input Schema
# -----------------
# Your input is a batch of images with shape (32, 240, 240, 3)
# We use -1 to indicate that the batch size can vary.
input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, IMG_SIZE, IMG_SIZE, 3), "image")])

# 2. Output Schema - Multilabel binary classification head
# ------------------
# Your model outputs a list of two arrays. We need to define a schema for each.
# Array 1: Shape (1, 3)
output_schema_array1 = TensorSpec(np.dtype(np.float32), (-1, 3), "classification")

# Array 2: Shape (1, 3, 4) - 3 Bounding boxes per classification 
output_schema_array2 = TensorSpec(np.dtype(np.float32), (-1, 3, 4), "bounding_box")

# Create a schema for the list of outputs
output_schema = Schema([output_schema_array1, output_schema_array2])

# 3. Model Signature
# --------------------
# Combine the input and output schemas into a ModelSignature
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

In [23]:
import os
import mlflow
# to_monitor = 'val_classification_AUC'
# mode = 'max'
to_monitor = 'val_bounding_box_mse'
mode = 'min'
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, 
                                            patience=5, 
                                            monitor=to_monitor,
                                            mode=mode,
                                            min_lr=1e-7,
                                            verbose=1),

    tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(str(CHECK_POINT_DIR), "detector_ckpt_{epoch}.keras") ,
                                        save_weights_only=False,
                                        save_best_only=True,
                                        monitor=to_monitor,
                                        mode=mode,
                                        verbose=1),
                                        
    tf.keras.callbacks.EarlyStopping(monitor=to_monitor, 
                                    patience=10,
                                    mode=mode, 
                                    restore_best_weights=True,
                                    verbose=1),

    ]

mlflow_exp = mlflow.set_experiment("/brain-tumor-resnet101-phase-training")

### Define Optimizer

## Model Building and Compilation

In [24]:
class MultiTaskLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.7, beta=0.3):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        
    def call(self, y_true, y_pred):
        cls_loss = _loss.set_binary_crossentropy_weighted_loss(
            positive_weights, negative_weights)(y_true[0], y_pred[0])
        box_loss = iou_loss.iou_loss(y_true[1], y_pred[1])
        return self.alpha * cls_loss + self.beta * box_loss

### Train and Validate the model

### Phase 1: Train feature extractor with classification head

In [25]:
run = mlflow.start_run() 
mlflow.tensorflow.autolog(log_models=True, 
                        log_datasets=False, 
                        log_input_examples=True,
                        log_model_signatures=True,
                        keras_model_kwargs={"save_format": "keras"},
                        checkpoint_monitor=to_monitor, 
                        checkpoint_mode=mode)

2025/03/24 07:56:28 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.18.0, but the installed version is 2.19.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.


In [26]:
from src.models.resnet101V2 import final_model

model = final_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES)

# Freeze regression branch and feature extractor
for layer in model.layers:
    if 'bounding_box' not in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False

In [27]:
# Only enable classification training
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=3e-4),
    loss={'classification': _loss.set_binary_crossentropy_weighted_loss(positive_weights, negative_weights),
          'bounding_box':  iou_loss.iou_loss},
    metrics={'classification': CLS_METRICS, 'bounding_box': REG_METRICS},
    loss_weights={'classification': 1.0, 'bounding_box': 0.0}
    )

#model.summary()

In [28]:
to_monitor = 'val_classification_loss'
mode = 'min'
phase1_epoch = 30
history = model.fit(
    train_ds,
    epochs=phase1_epoch,
    validation_data=valid_ds,
    batch_size=BATCH_SIZE,
    callbacks=[ 
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(str(CHECK_POINT_DIR), "phase1.weights.h5") ,
                                        save_weights_only=True,
                                        save_best_only=False,
                                        monitor=to_monitor,
                                        mode=mode,
                                        verbose=1),
        tf.keras.callbacks.EarlyStopping(monitor=to_monitor, 
                                    patience=3,
                                    mode=mode, 
                                    restore_best_weights=True,
                                    verbose=1),])

Epoch 1/30


I0000 00:00:1742803034.059453   49196 service.cc:152] XLA service 0x7387e8003310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742803034.059484   49196 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-03-24 07:57:14.913609: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742803041.297881   49196 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1742803072.565411   49196 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - bounding_box_iou_metric: 0.0026 - bounding_box_loss: 0.9974 - bounding_box_mae: 0.8207 - bounding_box_mse: 1.2125 - classification_AUC: 0.6552 - classification_f1_score: 0.5125 - classification_loss: 0.6330 - loss: 0.6817
Epoch 1: saving model to output/checkpoints/phase1.weights.h5


215/215 ━━━━━━━━━━━━━━━━━━━━ 145s 309ms/step - bounding_box_iou_metric: 0.0026 - bounding_box_loss: 0.9974 - bounding_box_mae: 0.8204 - bounding_box_mse: 1.2119 - classification_AUC: 0.6554 - classification_f1_score: 0.5126 - classification_loss: 0.6329 - loss: 0.6815 - val_bounding_box_iou_metric: 0.0025 - val_bounding_box_loss: 0.9975 - val_bounding_box_mae: 1.0915 - val_bounding_box_mse: 3.0550 - val_classification_AUC: 0.7779 - val_classification_f1_score: 0.3342 - val_classification_loss: 0.6862 - val_loss: 0.7095
Epoch 2/30
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - bounding_box_iou_metric: 0.0026 - bounding_box_loss: 0.9974 - bounding_box_mae: 0.7384 - bounding_box_mse: 1.0928 - classification_AUC: 0.7819 - classification_f1_score: 0.5473 - classification_loss: 0.5323 - loss: 0.5553
Epoch 2: saving model to output/checkpoints/phase1.weights.h5


215/215 ━━━━━━━━━━━━━━━━━━━━ 32s 114ms/step - bounding_box_iou_metric: 0.0026 - bounding_box_loss: 0.9974 - bounding_box_mae: 0.7384 - bounding_box_mse: 1.0927 - classification_AUC: 0.7819 - classification_f1_score: 0.5473 - classification_loss: 0.5322 - loss: 0.5553 - val_bounding_box_iou_metric: 0.0025 - val_bounding_box_loss: 0.9976 - val_bounding_box_mae: 0.7982 - val_bounding_box_mse: 1.3213 - val_classification_AUC: 0.7946 - val_classification_f1_score: 0.4933 - val_classification_loss: 0.5621 - val_loss: 0.5753
Epoch 3/30
214/215 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - bounding_box_iou_metric: 0.0022 - bounding_box_loss: 0.9978 - bounding_box_mae: 0.7929 - bounding_box_mse: 1.2315 - classification_AUC: 0.8455 - classification_f1_score: 0.5673 - classification_loss: 0.4529 - loss: 0.4678
Epoch 3: saving model to output/checkpoints/phase1.weights.h5
215/215 ━━━━━━━━━━━━━━━━━━━━ 29s 104ms/step - bounding_box_iou_metric: 0.0022 - bounding_box_loss: 0.9978 - bounding_box_mae: 0.7929 - bo

215/215 ━━━━━━━━━━━━━━━━━━━━ 31s 114ms/step - bounding_box_iou_metric: 0.0024 - bounding_box_loss: 0.9976 - bounding_box_mae: 0.8234 - bounding_box_mse: 1.3257 - classification_AUC: 0.8818 - classification_f1_score: 0.5983 - classification_loss: 0.3784 - loss: 0.3896 - val_bounding_box_iou_metric: 0.0013 - val_bounding_box_loss: 0.9987 - val_bounding_box_mae: 0.7470 - val_bounding_box_mse: 1.0330 - val_classification_AUC: 0.8290 - val_classification_f1_score: 0.5789 - val_classification_loss: 0.5301 - val_loss: 0.5359
Epoch 5/30
214/215 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - bounding_box_iou_metric: 0.0021 - bounding_box_loss: 0.9979 - bounding_box_mae: 0.8671 - bounding_box_mse: 1.4454 - classification_AUC: 0.9047 - classification_f1_score: 0.6107 - classification_loss: 0.3128 - loss: 0.3218
Epoch 5: saving model to output/checkpoints/phase1.weights.h5
215/215 ━━━━━━━━━━━━━━━━━━━━ 31s 110ms/step - bounding_box_iou_metric: 0.0021 - bounding_box_loss: 0.9979 - bounding_box_mae: 0.8671 - bo

2025/03/24 08:04:21 WARNING mlflow.models.signature: Failed to infer schema for outputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. To see the full traceback, set logging level to DEBUG.
2025/03/24 08:04:36 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpoz6yczjz/model, flavor: tensorflow). Fall back to return ['tensorflow==2.19.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/03/24 08:04:36 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


### Phase 2: Freeze classification head and train bounding box head

In [29]:
# Switch trainable components
for layer in model.layers:
    layer.trainable = False  # Freeze all first
    if 'bounding_box' in layer.name:
        layer.trainable = True
    if 'conv5_block3' in layer.name:  # Unfreeze deeper ResNet layers
        layer.trainable = True

In [32]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(1e-4),
    loss={'classification': _loss.set_binary_crossentropy_weighted_loss(positive_weights, negative_weights),
          'bounding_box':  iou_loss.iou_loss},
    metrics={'classification': CLS_METRICS, 'bounding_box': REG_METRICS},
    loss_weights={'classification': 0.0, 'bounding_box': 1.0})

In [33]:
# Set the number of epochs for fine-tuning
to_monitor = 'val_bounding_box_iou_metric'
mode = 'max'
phase2_epoch = phase1_epoch + 40 

history = model.fit(
    train_ds,
    epochs=phase2_epoch,
    initial_epoch=history.epoch[-1],  # Start from the last epoch of initial training
    validation_data=valid_ds,
    batch_size=BATCH_SIZE,
    callbacks=[ 
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, 
                                            patience=5, 
                                            monitor=to_monitor,
                                            mode=mode,
                                            min_lr=1e-7,
                                            verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(str(CHECK_POINT_DIR), "phase2.weights.h5") ,
                                        save_weights_only=True,
                                        save_best_only=False,
                                        monitor=to_monitor,
                                        mode=mode,
                                        verbose=1),
        tf.keras.callbacks.EarlyStopping(monitor=to_monitor, 
                                    patience=5,
                                    mode=mode, 
                                    restore_best_weights=True,
                                    verbose=1),])

Epoch 70/70


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - bounding_box_iou_metric: 0.3955 - bounding_box_loss: 0.6045 - bounding_box_mae: 0.2774 - bounding_box_mse: 0.1902 - classification_AUC: 0.9328 - classification_f1_score: 0.5638 - classification_loss: 0.5354 - loss: 0.6045
Epoch 70: saving model to output/checkpoints/phase2.weights.h5
215/215 ━━━━━━━━━━━━━━━━━━━━ 42s 105ms/step - bounding_box_iou_metric: 0.3955 - bounding_box_loss: 0.6045 - bounding_box_mae: 0.2774 - bounding_box_mse: 0.1902 - classification_AUC: 0.9328 - classification_f1_score: 0.5638 - classification_loss: 0.5354 - loss: 0.6045 - val_bounding_box_iou_metric: 0.2620 - val_bounding_box_loss: 0.7384 - val_bounding_box_mae: 0.2890 - val_bounding_box_mse: 0.1930 - val_classification_AUC: 0.8818 - val_classification_f1_score: 0.5485 - val_classification_loss: 0.5880 - val_loss: 0.7381 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 70.


2025/03/24 08:23:31 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: Changing param values is not allowed. Param with key='epochs' was already logged with value='30' for run ID='f0c78cc579984d95a3fb284db4a43dc0'. Attempted logging new value '70'.


### Phase 3: Fine-tune entire model with reduced learning rate

In [37]:
# Unfreeze all layers except first 150
for layer in model.layers[:150]:
    layer.trainable = False
for layer in model.layers[150:]:
    layer.trainable = True

In [38]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW( learning_rate=1e-5, 
    clipnorm=1.0  # Essential for stable training
    ),
    loss={'classification': _loss.set_binary_crossentropy_weighted_loss(positive_weights, negative_weights),
          'bounding_box':  iou_loss.iou_loss},
    metrics={'classification': CLS_METRICS, 'bounding_box': REG_METRICS},
    # Train with 0 weight for classification
    loss_weights={'classification': 0.1,  'bounding_box': 0.9 } )

In [39]:
phase3_epoch = phase2_epoch + 40 

history = model.fit(
    train_ds,
    epochs=phase3_epoch,
    initial_epoch=history.epoch[-1],  # Start from the last epoch of initial training
    validation_data=valid_ds,
    batch_size=BATCH_SIZE,
    callbacks=[callbacks])

Epoch 70/110


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - bounding_box_iou_metric: 0.3356 - bounding_box_loss: 0.6644 - bounding_box_mae: 0.2778 - bounding_box_mse: 0.1945 - classification_AUC: 0.9861 - classification_f1_score: 0.6272 - classification_loss: 0.0829 - loss: 0.6132
Epoch 70: val_bounding_box_mse did not improve from 0.18616
215/215 ━━━━━━━━━━━━━━━━━━━━ 102s 211ms/step - bounding_box_iou_metric: 0.3356 - bounding_box_loss: 0.6644 - bounding_box_mae: 0.2778 - bounding_box_mse: 0.1945 - classification_AUC: 0.9861 - classification_f1_score: 0.6272 - classification_loss: 0.0829 - loss: 0.6132 - val_bounding_box_iou_metric: 0.2634 - val_bounding_box_loss: 0.7366 - val_bounding_box_mae: 0.2893 - val_bounding_box_mse: 0.1970 - val_classification_AUC: 0.9099 - val_classification_f1_score: 0.5888 - val_classification_loss: 0.4350 - val_loss: 0.7134 - learning_rate: 1.0000e-05
Epoch 71/110
214/215 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - bounding_box_iou_metric: 0.3449 - bounding_box_loss: 0.6556 - b

2025/03/24 08:38:14 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: Changing param values is not allowed. Param with key='epochs' was already logged with value='30' for run ID='f0c78cc579984d95a3fb284db4a43dc0'. Attempted logging new value '110'.


In [ ]:
mlflow.tensorflow.log_model(
model,
"my_model",
signature=signature,
code_paths=["src/losses"])

In [ ]:
raise Exception("No GPU found")

In [ ]:
# mlflow.tensorflow.log_model(
#     model,
#     "my_model",
#     signature=signature,
#     code_paths=["src/losses"],
# )

In [ ]:
model_uri: str = "runs:/{}/model".format(run.info.run_id)
loaded_model = mlflow.tensorflow.load_model(model_uri)

loaded_model.evaluate(valid_ds, return_dict=True)

## Model Evaluation

### Testing Datasets setup

In [ ]:
import matplotlib.pyplot as plt

def visualize_training_results(history):
    """
    Visualizes training and validation loss, and training and validation accuracy.

    Args:
        history: A dictionary or object containing training history data.
                 For example, a Keras History object or a dictionary with keys:
                 'loss', 'val_loss', 'accuracy', 'val_accuracy'.
    """

    if isinstance(history, dict):
        # Assumes history is a dictionary
        loss = history.get('loss')
        val_loss = history.get('val_loss')
        accuracy = history.get('accuracy')
        val_accuracy = history.get('val_accuracy')
    else:
        # Assumes history is a Keras History object or similar
        loss = history.history.get('loss')
        val_loss = history.history.get('val_loss')
        accuracy = history.history.get('accuracy')
        val_accuracy = history.history.get('val_accuracy')

    if loss and val_loss:
        epochs = range(1, len(loss) + 1)

        plt.figure(figsize=(12, 5))

        # Plot training & validation loss values
        plt.subplot(1, 2, 1)
        plt.plot(epochs, loss, 'r', label='Training loss')
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

    if accuracy and val_accuracy:
        if not (loss and val_loss):
          plt.figure(figsize=(12, 5))
        else:
          plt.subplot(1, 2, 2)
        # Plot training & validation accuracy values
        plt.plot(epochs, accuracy, 'r', label='Training accuracy')
        plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

    plt.tight_layout() #prevents overlapping titles/labels
    plt.show()


In [ ]:
visualize_training_results(history.history)

In [ ]:
prepare_test_dataset = AnnotationProcessor(annotation_file= str(TEST_DIR/'_annotations.csv'))
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
test_image_paths, test_class_ids, test_bboxes = prepare_test_dataset.process_annotations(image_dir=TEST_DIR, class_id_map=_class_map)

len(test_image_paths), len(test_class_ids), len(test_bboxes)

In [ ]:
test_dl = DataLoader(test_image_paths, test_class_ids, test_bboxes, img_size=IMG_SIZE)
test_ds = test_dl.load_val_dataset()
y_true_labels = test_dl.multi_hot_class_ids
y_true_bboxes = test_dl.padded_bbx
test_ds = Preprocessor(test_ds).preprocess()
test_ds = test_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

In [ ]:
results = model.evaluate(test_ds, return_dict=True, steps=1)
print("Testing accuracy: ", results)

In [ ]:
results

In [ ]:
from sklearn.metrics import classification_report

y_prob_pred, pred_bbx = model.predict(test_ds)
y_prob_pred[0], pred_bbx[0]

In [ ]:
y_pred = (y_prob_pred>0.5).astype(int)
y_pred

In [ ]:
report = classification_report(y_true_labels, y_pred, labels=[0,1,2], target_names=CLASS_NAME)
print(report)

In [ ]:
from src.utils.visualization_funcs import plot_auc_curve


plot_auc_curve(OUTPUT_DIR, CLASS_NAME, y_true_labels, y_prob_pred)

In [ ]:
test_bboxes

In [ ]:
pred_bbx

In [ ]:
from src.losses.iou_loss import iou_metric
def plot_iou_histogram(y_true_bbox, y_pred_bbox, class_ids):
    """
    Plots a histogram of Intersection over Union (IoU) scores.

    Args:
        y_true_bbox: Ground truth bounding boxes (list of lists or numpy array).
        y_pred_bbox: Predicted bounding boxes (list of lists or numpy array).
        class_ids: list of class ids.
    """
    fig, axs = plt.subplots(1)

    iou_scores = iou_metric(y_true_bbox, y_pred_bbox)

    # fig.figure(figsize=(10, 6))
    axs.hist(iou_scores, bins=20, range=(0, 1), edgecolor='black')
    axs.set_title('IoU Score Distribution')
    axs.set_xlabel('IoU Score')
    axs.set_ylabel('Frequency')
    axs.grid(True)
    plt.show()
    plt.savefig(f"{OUTPUT_DIR}/iou_histogram.png")
    return fig


In [ ]:
fig = plot_iou_histogram(y_true_bboxes, pred_bbx, pred_bbx)
mlflow.log_figure(fig, 'iou_histogram.png')